---
# 무관심 가사 20개 Markov Chain 하기

In [1]:
import pandas as pd

In [2]:
ballad = pd.read_csv("../../Data/ballad_train60.csv")

In [3]:
ballad[ballad.감정 == "regret"]

,Unnamed: 0,가수명,노래제목,발매일,가사,좋아요수,감정
40,40,어반자카파,아직도 나를 사랑한다면,2016-05-27,노래를 불러요 우릴 헤어지게 만든 그 노래를 이 일만 끝나면 더 잘할 수 있다고 그...,7655,regret
41,41,수지,I Love You Boy,2017-10-11,그댈 보기 위해 먼 길을 건너왔네요 힘든 길인걸 알면서도 그저 걸었죠 I fly f...,33264,regret
42,42,박새별,사랑이 우릴 다시 만나게 한다면,2013-05-28,사랑이 우릴 다시 만나게 해준다면 나 그대에게 해주고 싶은 말이 너무 많아요 그댈 ...,12564,regret
43,43,폴킴,사랑 알 수 없나봐,2018-12-16,사랑 알 수 없나 봐 어떤 마음인지 자꾸 의심하게 돼 청개구리처럼 반대로 하게 되는...,41914,regret
44,44,길구봉구,"다시, 우리",2018-05-24,뚜루루 온종일 비가 내렸지 눅눅히 젖은 신발을 보다가 엉겨 붙어버린 기억의 먼지 한...,33582,regret
45,45,소유 (SOYOU),길에서 (Feat. 적재),2019-12-23,하루 종일 비가 와서 니 얼굴이 또 생각 나 눈물이 났어 한 걸음 더 걸어가면 잊혀...,5946,regret
46,46,T.P RETRO (타디스 프로젝트舊),오늘도 그대만 (Feat. 정동원),2017-02-28,봄날처럼 몰래 다가왔던 나의 그대 여름 같이 뜨거웠던 우리의 시간들 어느덧 찾아온 ...,101735,regret
47,47,박효신,별 시 (別 時),2018-04-30,You are my star 이 노래는 너야 불빛 하나 없는 길을 걷게 해 줄 기나...,61064,regret
48,48,장나라,그게 정말이니?,2003-12-03,후회는 없을거라고 그만큼 사랑했다고 지금 흔들리는 목소리로 이별을 얘기하는거니 사랑...,9600,regret
49,49,이진성 (먼데이 키즈),Letter,2009-02-12,텅 빈 방안에 앉아 너의 사진을 본다 해맑았던 미소가 날 붙잡아 발이 떨어지질 않아...,9966,regret


In [4]:
ballad_60 = ballad.drop(index=range(40,60),inplace=True)

In [5]:
ballad_60

In [6]:
ballad_lyrics_indifference = ballad[ballad.감정 == 'indifference'].가사

In [7]:
ballad_lyrics_indifference= pd.DataFrame(ballad_lyrics_indifference)
ballad_lyrics_indifference.to_csv("../../Data/ballad_lyrics_indifference.csv",index=False)

NameError: name 'ballad_lyrics_indifference' is not defined

In [ ]:
import markov
from konlpy.tag import Okt
import json

In [ ]:
ballad_file = "../../Note/Markov/ballad_indifference.txt"
dict_file = "../../Note/Markov/markov_ballad_indifference.json"

In [ ]:
file = open("../../Data/ballad_lyrics_indifference.csv",encoding='UTF-8')
text = str(file.readlines())
text

In [ ]:
text = text.split(",")[1:]

In [ ]:
total = ""
for i in range(len(text)):
     total += " " + text[i] + "." 

In [ ]:
len(total)

In [ ]:
total= total.rstrip("].")

In [ ]:
total += "."

In [ ]:
total

In [ ]:
# 텍스트를 한 줄씩 처리하기
twitter = Okt()
lines = total.split("\n")
for line in lines:
    malist = twitter.pos(line,norm=True,stem=False)
    words = []
    for word in malist:
        if not word[1] in ['Punctuation']: # .(마침표)제외한 ? "" ! 를 사용하지 않음
            words.append(word[0])
        if word[0] == ". ":
            words.append(word[0])

In [ ]:
# 딕셔너리 생성
dic = markov.make_dic(words)
json.dump(dic,open(dict_file,"w",encoding="utf-8"))

In [ ]:
dic

In [ ]:
# # 문장 만들기
# for i in range(2):
#     s = markov.make_sentence(dic)
#     print(s)
#     print("-------")

In [ ]:
dic.keys()

In [ ]:
dic.values()